<a href="https://colab.research.google.com/github/robgon-art/faster-whisper-test/blob/main/CTranslate2_Whisper_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ctranslate2 transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.9/31.9 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 1.9 MB/s eta 0:00:00


In [2]:
!gdown 1xbZ6MTAFukAhqEXqQEQi-ZhY1us0tFwy

Downloading...
From: https://drive.google.com/uc?id=1xbZ6MTAFukAhqEXqQEQi-ZhY1us0tFwy
To: /content/CoreyTInterview-32KHz_180s.wav
100% 11.5M/11.5M [00:00<00:00, 26.9MB/s]


In [3]:
!ct2-transformers-converter --model openai/whisper-small --output_dir whisper-small-ct2
!ct2-transformers-converter --model openai/whisper-small --output_dir whisper-small-ct2-int8 --quantization int8

In [4]:
import binascii
def crc32_checksum(input_string):
    input_bytes = input_string.encode() if isinstance(input_string, str) else input_string
    checksum = binascii.crc32(input_bytes)
    return checksum & 0xFFFFFFFF

In [5]:
import nltk
import difflib
def different_words(str1, str2):
    words1 = str1.split()
    words2 = str2.split()
    matcher = difflib.SequenceMatcher(None, words1, words2)
    differences = []
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == 'replace':
            max_len = max(a1 - a0, b1 - b0)
            for i in range(max_len):
                a = words1[a0 + i] if a0 + i < a1 else 'NA'
                b = words2[b0 + i] if b0 + i < b1 else 'NA'
                differences.append((a, b))
        elif opcode == 'delete':
            for a in words1[a0:a1]:
                differences.append((a, 'NA'))
        elif opcode == 'insert':
            for b in words2[b0:b1]:
                differences.append(('NA', b))
    return differences

In [6]:
import ctranslate2
import librosa
import transformers
import textwrap
import time

file_name = "/content/CoreyTInterview-32KHz_180s.wav" # language: en

# Load and resample the audio file.
audio, _ = librosa.load(file_name, sr=16000, duration=30)

# Compute the features of the first 30 seconds of audio.
processor = transformers.WhisperProcessor.from_pretrained("openai/whisper-small")
inputs = processor(audio, return_tensors="np", sampling_rate=16000)
features = ctranslate2.StorageView.from_array(inputs.input_features)

# Describe the task in the prompt.
# See the prompt format in https://github.com/openai/whisper.
prompt = processor.tokenizer.convert_tokens_to_ids(
    [
        "<|startoftranscript|>",
        "<|en|>",
        "<|transcribe|>",
        "<|notimestamps|>",  # Remove this token to generate timestamps.
    ]
)

In [7]:
# Load the int8 model on CPU.
model = ctranslate2.models.Whisper("whisper-small-ct2-int8")

# Detect the language.
results = model.detect_language(features)
language, probability = results[0][0]
print("Detected language: %s with probability: %f" % (language, probability))

# Show the encoder results
encoder_results = model.encode(features)
print("Encoder results:", encoder_results)

# Run generation for the 30-second window.
t1 = time.time()
results = model.generate(features, [prompt], beam_size=1, sampling_temperature=0)
transcription_1 = processor.decode(results[0].sequences_ids[0]).strip()
t2= time.time()
print("Total Time:", round(t2 - t1, 2), "seconds")
checksum_1 = crc32_checksum(transcription_1)
print("Trnascription Checksum:", checksum_1)
print(textwrap.fill(transcription_1, width=120))

Detected language: <|en|> with probability: 0.997071
Encoder results:  -0.774333 -1.40021 1.49652 ... -1.80854 -0.702311 0.326999
[cpu:0 float32 storage viewed as 1x1500x768]
Total Time: 9.57 seconds
Trnascription Checksum: 2912471593
Okay, you can still get set up. I'm just gonna do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
the Post Cafe. I'm Corey Tedrow, your host. Today we're gonna be talking about long distance tech support. And we are
joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome,
Susan.


In [8]:
# Results of runs on difference instances of the same type of CPU:

# Detected language: <|en|> with probability: 0.997174
# Encoder results:  -0.795277 -1.39131 1.50737 ... -1.83052 -0.697069 0.322911
# [cpu:0 float32 storage viewed as 1x1500x768]
# Total Time: 8.29 seconds
# Trnascription Checksum: 1060283712
# Okay, you can still get set up. I'm just gonna do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
# the Post Cafe. I'm Corey Tedrow, your host. Today we're gonna be talking about long distance tech support. And we are
# joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome
# Susan.

# Detected language: <|en|> with probability: 0.997071                          <-- note differences
# Encoder results:  -0.774333 -1.40021 1.49652 ... -1.80854 -0.702311 0.326999  <-- here, too
# [cpu:0 float32 storage viewed as 1x1500x768]
# Total Time: 9.57 seconds
# Trnascription Checksum: 2912471593
# Okay, you can still get set up. I'm just gonna do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
# the Post Cafe. I'm Corey Tedrow, your host. Today we're gonna be talking about long distance tech support. And we are
# joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome, <-- note comma
# Susan.

In [9]:
# Load the int8 model on GPU.
model = ctranslate2.models.Whisper("whisper-small-ct2-int8", device="cuda")

# Detect the language.
results = model.detect_language(features)
language, probability = results[0][0]
print("Detected language: %s with probability: %f" % (language, probability))

# Show the encoder results
encoder_results = model.encode(features)
print("Encoder results:", encoder_results)

# Run generation for the 30-second window.
t1 = time.time()
results = model.generate(features, [prompt], beam_size=1, sampling_temperature=0)
transcription_2 = processor.decode(results[0].sequences_ids[0]).strip()
t2= time.time()
print("Total Time:", round(t2 - t1, 2), "seconds")
checksum_2 = crc32_checksum(transcription_2)
print("Trnascription Checksum:", checksum_2)
print(textwrap.fill(transcription_2, width=120))

Detected language: <|en|> with probability: 0.997335
Encoder results:  -0.811487 -1.45845 1.50149 ... -1.84109 -0.710835 0.246928
[cuda:0 float32 storage viewed as 1x1500x768]
Total Time: 0.37 seconds
Trnascription Checksum: 1569094943
Okay. You can still get set up. I'm just going to do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
the Post Cafe. I'm Corey Tedrow, your host. Today we're going to be talking about long distance tech support. And we are
joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome,
Susan.


In [10]:
# Load the fp32 model on CPU.
model = ctranslate2.models.Whisper("whisper-small-ct2")

# Detect the language.
results = model.detect_language(features)
language, probability = results[0][0]
print("Detected language: %s with probability: %f" % (language, probability))

# Show the encoder results
encoder_results = model.encode(features)
print("Encoder results:", encoder_results)

# Run generation for the 30-second window.
t1 = time.time()
results = model.generate(features, [prompt])
transcription_3 = processor.decode(results[0].sequences_ids[0]).strip()
t2= time.time()
print("Total Time:", round(t2 - t1, 2), "seconds")
checksum_3 = crc32_checksum(transcription_3)
print("Trnascription Checksum:", checksum_3)
print(textwrap.fill(transcription_3, width=120))

Detected language: <|en|> with probability: 0.997852
Encoder results:  -0.724928 -1.39428 1.49868 ... -1.90465 -0.740848 0.322737
[cpu:0 float32 storage viewed as 1x1500x768]
Total Time: 18.17 seconds
Trnascription Checksum: 2809285841
Okay. You can still get set up. I'm just going to do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
the post cafe. I'm Corey Tedrow, your host. Today we're going to be talking about long distance tech support. And we are
joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome,
Susan.


In [11]:
# Load the fp32 model on GPU.
model = ctranslate2.models.Whisper("whisper-small-ct2", device="cuda")

# Detect the language.
results = model.detect_language(features)
language, probability = results[0][0]
print("Detected language: %s with probability: %f" % (language, probability))

# Show the encoder results
encoder_results = model.encode(features)
print("Encoder results:", encoder_results)

# Run generation for the 30-second window.
t1 = time.time()
results = model.generate(features, [prompt])
transcription_4 = processor.decode(results[0].sequences_ids[0]).strip()
t2= time.time()
print("Total Time:", round(t2 - t1, 2), "seconds")
checksum_4 = crc32_checksum(transcription_4)
print("Trnascription Checksum:", checksum_4)
print(textwrap.fill(transcription_4, width=120))

Detected language: <|en|> with probability: 0.997853
Encoder results:  -0.724926 -1.39429 1.49868 ... -1.90464 -0.740849 0.322737
[cuda:0 float32 storage viewed as 1x1500x768]
Total Time: 0.8 seconds
Trnascription Checksum: 2809285841
Okay. You can still get set up. I'm just going to do my intros. Don't laugh at me. Okay. All right. Hello and welcome to
the post cafe. I'm Corey Tedrow, your host. Today we're going to be talking about long distance tech support. And we are
joined by Susan Rayborn Miller. She is the technology manager at the production hive in Knoxville, Tennessee. Welcome,
Susan.


In [12]:
print("checksum_1", checksum_1)
print("checksum_2", checksum_2)
print("checksum_3", checksum_3)
print("checksum_4", checksum_4)

checksum_1 1060283712
checksum_2 1569094943
checksum_3 2809285841
checksum_4 2809285841


In [13]:
differences = different_words(transcription_1, transcription_2)
if differences:
    print("Different word pairs:")
    for word1, word2 in differences:
        print(f"('{word1}', '{word2}')")
else:
    print("No different words found")

Different word pairs:
('Okay,', 'Okay.')
('you', 'You')
('gonna', 'going')
('NA', 'to')
('gonna', 'going')
('NA', 'to')
('Welcome', 'Welcome,')


In [14]:
differences = different_words(transcription_1, transcription_3)
if differences:
    print("Different word pairs:")
    for word1, word2 in differences:
        print(f"('{word1}', '{word2}')")
else:
    print("No different words found")

Different word pairs:
('Okay,', 'Okay.')
('you', 'You')
('gonna', 'going')
('NA', 'to')
('Post', 'post')
('Cafe.', 'cafe.')
('gonna', 'going')
('NA', 'to')
('Welcome', 'Welcome,')


In [15]:
differences = different_words(transcription_2, transcription_3)
if differences:
    print("Different word pairs:")
    for word1, word2 in differences:
        print(f"('{word1}', '{word2}')")
else:
    print("No different words found")

Different word pairs:
('Post', 'post')
('Cafe.', 'cafe.')


In [16]:
differences = different_words(transcription_3, transcription_4)
if differences:
    print("Different word pairs:")
    for word1, word2 in differences:
        print(f"('{word1}', '{word2}')")
else:
    print("No different words found")

No different words found
